# Importing the libraries

In [1]:
from pandasdmx import Request

In [2]:
import pandas as pd

In [3]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet

In [4]:
output_notebook()

Loading BokehJS ...

# Connecting to the webservice

In [5]:
estat = Request('ESTAT')

### Downloading dataflow definitions

In [6]:
dflow = estat.dataflow()

## Defining functions

### Data structure

In [7]:
def dsd_resp(id):
    """
    The function takes a dataflow identifier and requests from the Eurostat SDMX api the dataflow's datastructure, datastructure id and datastructure response.
    """    
    # Defining the data set's dataflow based on the data set id.
    df = dflow.dataflow[id]
    
    # Printing the table's name
    print("The table's name:")
    print(dflow.write().dataflow.loc[id][0])
    
    # Acquiring the dataflow's datastructure id
    dsd_id = df.structure.id
    
    # Requesting for the dataflow's datastructure
    dsd_resp = estat.get(resource_type = 'datastructure', resource_id = dsd_id)
    
    # Requesting the dataflow's datastructure
    dsd = dsd_resp.datastructure[dsd_id]
    
    return dsd_resp, dsd_id, dsd

### Datastructure attributes

In [8]:
def dsd_att(dsd):
    """The function takes a table datastructure definition and prints its measure, dimension and attribute lists."""
    # Measures
    print("Measures:")
    print(dsd.measures.aslist())
    
    # Dimensions
    print("\nDimensions:")
    print(dsd.dimensions.aslist())
    
    # Attributes
    print("\nAttributes:")
    print(dsd.attributes.aslist())

## Selecting the tables

### Listing the tables from the 'High-tech industry and knowledge-intensive services (**htec**)' database

In [9]:
dflow.write().dataflow[dflow.write().dataflow.index.str.startswith('htec') == True]

,name
dataflow,
htec_cis3,Innovation in high-tech sectors in SMEs (CIS3)...
htec_cis4,"Innovation in high-tech sectors (CIS 2004), EU..."
htec_cis5,"Innovation in high-tech sectors (CIS 2006), EU..."
htec_cis6,"Innovation in high-tech sectors (CIS 2008, CIS..."
htec_eco_ent,Enterprises in high-tech sectors by NACE Rev.1...
htec_eco_ent2,Enterprises in high-tech sectors by NACE Rev.2...
htec_eco_sbs,Economic data in high-tech sectors by NACE Rev...
htec_eco_sbs2,Economic data in high-tech sectors by NACE Rev...
htec_emp_nat,Employment in technology and knowledge-intensi...


### The tables I am going to analyze

In [10]:
id1 = 'htec_sti_exp2'

In [11]:
dflow.write().dataflow.loc[id1][0]

'Business enterprise R&D expenditure in high-tech sectors - NACE Rev. 2'

In [12]:
id2 = 'htec_vci_stage2'

In [13]:
dflow.write().dataflow.loc[id2][0]

'Venture capital investment by detailed stage of development (from 2007, source: EVCA)'

# Venture capital data

### Getting the table's dataflow definition and datastructure

In [14]:
dsd_resp2, dsd_id2, dsd2 = dsd_resp(id2)

The table's name:
Venture capital investment by detailed stage of development (from 2007, source: EVCA)


### Downloading the table's datastructure definition

In [15]:
dsd_att(dsd2)

Measures:
[PrimaryMeasure | OBS_VALUE]

Dimensions:
[Dimension | FREQ, Dimension | EXPEND, Dimension | UNIT, Dimension | GEO, TimeDimension | TIME_PERIOD]

Attributes:
[DataAttribute | OBS_FLAG, DataAttribute | OBS_STATUS]


Codelist:

In [16]:
dsd_resp2.write().codelist.loc[:,:] # [['EXPEND', 'UNIT'],:]

dim_or_attr  \
FREQ       FREQ                   D   
           A                      D   
           D                      D   
           H                      D   
           M                      D   
           Q                      D   
           S                      D   
           W                      D   
EXPEND     EXPEND                 D   
           INV                    D   
           INV_BUY                D   
           INV_GROW               D   
           INV_REPL               D   
           INV_RESC               D   
           INV_VEN                D   
           INV_VEN_LATE           D   
           INV_VEN_SEED           D   
           INV_VEN_STAR           D   
UNIT       UNIT                   D   
           MIO_EUR                D   
           NR_COMP                D   
           PC_GDP                 D   
GEO        GEO                    D   
           AT                     D   
           BE                     D   
           BG                     D   
           CH                     D   
           CZ                     D   
           DE                     D   
           DK                     D   
...                             ...   
           FI                     D   
           FR                     D   
           HU                     D   
           IE                     D   
           IT                     D   
           LU                     D   
           NL                     D   
           NO                     D   
           PL                     D   
           PT                     D   
           RO                     D   
           SE                     D   
           UK                     D   
OBS_FLAG   OBS_FLAG               A   
           b                      A   
           c                      A   
           d                      A   
           e                      A   
           f                      A   
           i                      A   
           n                      A   
           p                      A   
           r                      A   
           s                      A   
           u                      A   
           z                      A   
OBS_STATUS OBS_STATUS             A   
           -                      A   
           0                      A   
           na                     A   

                                                                      name  
FREQ       FREQ                                                       FREQ  
           A                                                        Annual  
           D                                                         Daily  
           H                                                     Half-year  
           M                                                       Monthly  
           Q                                                     Quarterly  
           S                                                   Semi-annual  
           W                                                        Weekly  
EXPEND     EXPEND                                                   EXPEND  
           INV                                                  Investment  
           INV_BUY                                   Investment for buyout  
           INV_GROW                             Investment in growth stage  
           INV_REPL                             Investment for replacement  
           INV_RESC                       Investment for rescue/turnaround  
           INV_VEN       Venture capital investment (seed, start-up and...  
           INV_VEN_LATE                          Investment in later stage  
           INV_VEN_SEED                           Investment in seed stage  
           INV_VEN_STAR                       Investment in start-up stage  
UNIT       UNIT                                                       UNIT  
           MIO_EUR                                            Mil

### Requesting the data

In [17]:
dresp2 = estat.get(resource_type = 'data',
                   resource_id = id2,
                   params = {'references': None,
                             #'startPeriod': '2014',
                             #'endPeriod': '2014'
                            })

In [18]:
dat2 = dresp2.data

In [19]:
dat2.dim_at_obs

'TIME_PERIOD'

In [20]:
ser2 = list(dat2.series)
len(ser2)

621

In [21]:
ser2[5].key

SeriesKey(UNIT='MIO_EUR', EXPEND='INV', GEO='DE', FREQ='A')

In [22]:
set(s.key.UNIT for s in dat2.series)

{'MIO_EUR', 'NR_COMP', 'PC_GDP'}

### Writing the data into DataFrame

In [23]:
iter2 = (s for s in dat2.series if (s.key.GEO in ['EU15']) == False)
tab2 = dresp2.write(iter2)
tab2 = tab2.stack('GEO')
tab2.columns = tab2.columns.droplevel(['FREQ']) # Dropping 'FREQ' and 'UNIT' levels
tab2.swaplevel('UNIT', 'EXPEND', axis = 1)
tab2 = tab2.loc['2015', ['PC_GDP', 'MIO_EUR']]
# tab2.dropna(axis = 0, how = 'any', inplace = True)
tab2.index = tab2.index.droplevel(0) # Dropping the 'TIME_PERIOD' axis
tab2

UNIT    MIO_EUR                                                           \
EXPEND      INV  INV_BUY INV_GROW INV_REPL INV_RESC INV_VEN INV_VEN_LATE   
GEO                                                                        
AT        109.0     27.0     54.0      0.0      0.0    27.0          5.0   
BE        722.0    562.0     68.0     18.0     11.0    63.0         30.0   
BG         16.0     11.0      5.0      0.0      0.0     1.0          0.0   
CH        640.0    197.0    107.0     27.0     26.0   283.0        143.0   
CZ         13.0      0.0      9.0      0.0      0.0     4.0          1.0   
DE       5996.0   4515.0    554.0    165.0      6.0   757.0        330.0   
DK       1162.0    832.0     35.0      5.0      0.0   290.0         53.0   
EL          0.0      0.0      0.0      0.0      0.0     0.0          0.0   
ES       1051.0    667.0    152.0     81.0     45.0   105.0         40.0   
FI        512.0    371.0     40.0      2.0      1.0    98.0         25.0   
FR       9451.0   6848.0   1743.0    112.0      3.0   744.0        364.0   
HU        118.0     94.0      0.0      0.0      0.0    24.0          3.0   
IE        161.0     64.0      9.0      0.0      0.0    89.0         38.0   
IT       1160.0    974.0    123.0     30.0      0.0    33.0         13.0   
LU         74.0     32.0      0.0      0.0      0.0    41.0          7.0   
NL       1708.0   1031.0    388.0     46.0     23.0   220.0         64.0   
NO       1165.0    730.0    344.0      0.0      0.0    92.0         53.0   
PL        802.0    653.0    115.0      5.0      0.0    29.0         15.0   
PT        150.0     40.0     27.0     13.0      0.0    70.0         18.0   
RO         50.0     46.0      2.0      0.0      0.0     1.0          1.0   
SE       1722.0   1429.0    106.0     37.0      0.0   150.0         57.0   
UK      20521.0  17382.0   2157.0     67.0     82.0   833.0        512.0   

UNIT                             PC_GDP                                     \
EXPEND INV_VEN_SEED INV_VEN_STAR    INV INV_BUY INV_GROW INV_REPL INV_RESC   
GEO                                                                          
AT             10.0         13.0  0.032     NaN      NaN      NaN      NaN   
BE              0.0         33.0  0.176     NaN      NaN      NaN      NaN   
BG              0.0          1.0  0.036     NaN      NaN      NaN      NaN   
CH              5.0        134.0    NaN     NaN      NaN      NaN      NaN   
CZ              1.0          1.0  0.008     NaN      NaN      NaN      NaN   
DE             44.0        383.0  0.198     NaN      NaN      NaN      NaN   
DK              8.0        230.0  0.437     NaN      NaN      NaN      NaN   
EL              0.0          0.0  0.000     NaN      NaN      NaN      NaN   
ES              3.0         62.0  0.097     NaN      NaN      NaN      NaN   
FI              9.0         64.0  0.247     NaN      NaN      NaN      NaN   
FR             11.0        370.0  0.433     NaN      NaN      NaN      NaN   
HU              3.0         18.0  0.109     NaN      NaN      NaN      NaN   
IE              0.0         50.0  0.075     NaN      NaN      NaN      NaN   
IT              2.0         18.0  0.071     NaN      NaN      NaN      NaN   
LU              0.0         34.0  0.141     NaN      NaN      NaN      NaN   
NL              8.0        147.0  0.252     NaN      NaN      NaN      NaN   
NO              2.0         37.0  0.333     NaN      NaN      NaN      NaN   
PL              3.0         12.0  0.188     NaN      NaN      NaN      NaN   
PT              5.0         47.0  0.084     NaN      NaN      NaN      NaN   
RO              0.0          0.0  0.031     NaN      NaN      NaN      NaN   
SE              0.0         93.0  0.388     NaN      NaN      NaN      NaN   
UK             10.0        310.0  0.799     NaN      NaN      NaN      NaN   

UNIT                                                   
EXPEND INV_VEN INV_VEN_LATE INV_VEN_SEED INV_VEN_STAR  
GEO                                 

In [32]:
source = ColumnDataSource(data = dict(names = list(tab2.index), invest = tab2.loc[:,('PC_GDP','INV')], venture = tab2.loc[:,('PC_GDP','INV_VEN')]))

p = figure(plot_width = 600, plot_height = 600)
p.square(x = tab2.loc[:,('PC_GDP','INV')], y = tab2.loc[:,('PC_GDP','INV_VEN')], size = list(tab2.loc[:,('MIO_EUR','INV')] / 500), color = "blue", alpha = 0.6)

p.xaxis.axis_label = 'Total R&D Investment'
p.yaxis.axis_label = 'Venture Capital R&D Investment'

labels = LabelSet(x = 'invest',
                  y = 'venture',
                  text = 'names',
                  source = source,
                  render_mode='canvas',
                  level='glyph')

p.add_layout(labels)
show(p)